# Modeling
-------------

In modeling phase, various modeling techniques are selected and applied and their parameters are calibrated to optimal values. Typically, there are several techniques for the same data mining problem type. Some techniques have specific requirements on the form of data. Therefore, stepping back to the data preparation phase is often necessary.

- Modeling techuique selection: Selection the actual modeling technique that is to be used. 

- Test design generation: Generateion of a procedure or mechanism to test the model's quality and validity.  Split the dataset into train and test set, build the model on the train set and estimate its quality on the separate test set.

- Build model: Run the modeling tool on the prepared dataset to create one or more models.

- Assess model: Summarize results of this task, list qualities of generated models (e.g.,in terms of accuracy) and rank their quality in relation to each other. 


In [13]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Access the data

In [2]:
dataset = pd.read_parquet('..\data\preprocessed_data.parquet')
dataset

,review_id,product_category,total_votes,review,usefulness
41,R1FBO737KD9F2N,Electronics,23,Great noise cancelling headphones for the pric...,0.826087
145,R227GSNWI6BSZV,Electronics,20,"Garbage, lasted 8 months... warranty is useles...",1.000000
265,R4PF7S0TOV9S7,Electronics,17,A long lasting bluetooth sound bazooka!\nThis ...,0.882353
274,R22LKIOKMSOG8A,Electronics,13,nice!\nThis is a nice little turntable. Don't ...,0.923077
304,R3SJTYZBYBG4EE,Electronics,99,Very good charger for the price! But has a dow...,1.000000
...,...,...,...,...,...
9001932,R23WGBU0VIL6FA,Wireless,26,Great little radios\nI just purchased two of t...,1.000000
9001946,R1YWKS4FWD687C,Wireless,18,Sony FRS radio\nI have been a FRS radio user f...,1.000000
9001962,R2STV9N2M963YM,Wireless,24,"Great\nI have only had a short time, and I am ...",0.916667
9001989,R25BIFCRPWPUHA,Wireless,46,Excellent - bearing in mind the power/range li...,0.586957


## Simple preprocessing

In [3]:
dataset['review'] = dataset['review'].str.lower()
# remove stopwords 
stop = stopwords.words('english')
dataset['review'] = dataset['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
dataset['review'] = dataset['review'].str.replace('[^a-zA-Z\s]', '', regex=True)

## apply label encoding to the product_category column

In [4]:
label_encoder = LabelEncoder()
dataset['product_category'] = label_encoder.fit_transform(dataset['product_category'])

## Choose features and lable from datases

In [5]:
X = dataset[['product_category', 'total_votes', 'review']]

In [6]:
y = dataset['usefulness']

## Construct the column transformer

In [7]:
tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer(min_df=0.02)

column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', count_vectorizer, 'review'),
    ], 
    remainder='passthrough'
)

## Define the Decision Tree Regressor, Linear Regression, Gradient Boosting Regressor models

In [8]:
models = [
    ('Decision Tree Regressor', DecisionTreeRegressor()),
    ('Linear Regression', LinearRegression()),
    ('Gradient Boosting Regressor', GradientBoostingRegressor())
]

## Create a pipeline for each model

In [9]:
pipelines = []
for name, model in models:
    pipeline = Pipeline([
        ('preprocessor', column_transformer),
        ('model', model)
    ])
    pipelines.append((name, pipeline))

## Split data into train and test sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train and test models

In [14]:
for name, pipeline in pipelines:
    print(f'Training {name}')
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    
     # Save the trained model
    joblib.dump(pipeline, f'./models/{name}_model.pkl')
    print()

Training Decision Tree Regressor
Mean Squared Error: 0.11914937113486608

Training Linear Regression
Mean Squared Error: 0.06507291126904217

Training Gradient Boosting Regressor
Mean Squared Error: 0.060718931912111515

